<a href="https://colab.research.google.com/github/kmeng01/memit/blob/main/notebooks/memit.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

In [ ]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
cd /content && rm -rf /content/memit
git clone https://github.com/kmeng01/memit memit > install.log 2>&1
pip install -r /content/memit/scripts/colab_reqs/rome.txt >> install.log 2>&1
pip install --upgrade google-cloud-storage >> install.log 2>&1

In [ ]:
!nvidia-smi

Fri Jan 20 22:17:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    51W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Mass-Editing Memory in a Transformer
This notebook enables interactive experimentation with MEMIT and several other comparable baselines.
The goal is to write new facts (e.g. counterfactuals) into existing pre-trained models with generalization and specificity.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install transformers==4.25.1 accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 89.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.15.0
    Uninstalling transformers-4.15.0:
      Successfully uninstalled transformers-4.15.0


In [ ]:
# restart the notebook for most recent lib versions to load

In [ ]:
IS_COLAB = False
ALL_DEPS = False
try:
    import google.colab, torch, os

    IS_COLAB = True
    os.chdir("/content/memit")
    if not torch.cuda.is_available():
        raise Exception("Change runtime type to include a GPU.")
except ModuleNotFoundError as _:
    pass

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import GPTJForCausalLM
from transformers import GPTNeoXForCausalLM, GPTNeoXTokenizerFast


from util import nethook
from util.generate import generate_interactive, generate_fast

from experiments.py.demo import demo_model_editing, stop_execution

In [ ]:
import torch
torch.__version__

'1.13.1+cu116'

In [ ]:
import transformers
transformers.__version__

'4.25.1'

In [ ]:
import accelerate
accelerate.__version__

'0.15.0'

Here, you can specify a GPT model (`MODEL_NAME`).

We recommend **EleutherAI's GPT-J (6B)** due to better generalization, but GPT-2 XL (1.5B) consumes less memory.
* `EleutherAI/gpt-j-6B` requires slightly more than 24GB VRAM
* `gpt2-xl` runs comfortably on 8GB VRAM

In [ ]:
MODEL_NAME = "EleutherAI/gpt-neox-20b"
#MODEL_NAME = "EleutherAI/gpt-j-6B"

In [ ]:
MODEL_NAME 

'EleutherAI/gpt-neox-20b'

In [ ]:
model, tok = (
    
    #AutoModelForCausalLM.from_pretrained(
        #MODEL_NAME,
        #low_cpu_mem_usage=IS_COLAB,
        #torch_dtype=(None),
    #).to("cuda"),

    GPTJForCausalLM.from_pretrained(
        'EleutherAI/gpt-j-6B').half().to("cuda"), #  converts to fp16
    AutoTokenizer.from_pretrained('EleutherAI/gpt-j-6B'),

    # GPTNeoXForCausalLM.from_pretrained(
        # MODEL_NAME).half().to("cuda"),

    # GPTNeoXTokenizerFast.from_pretrained(MODEL_NAME)
    
)
tok.pad_token = tok.eos_token
model.config

Downloading:   0%|          | 0.00/617 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/57.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/926M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/910M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/910M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/910M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/910M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/604M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/620M [00:00<?, ?B/s]

OutOfMemoryError: ignored

A requested rewrite can be specified using `request`. `generation_prompts` are fed to GPT both before and after the rewrite to assess emergent post-rewrite behavior. See the bottom of this notebook for more examples.


In [ ]:
request = [
    {
        "prompt": "{} was the founder of",
        "subject": "Steve Jobs",
        "target_new": {"str": "Microsoft"},
    },
    {
        "prompt": "{} plays the sport of",
        "subject": "LeBron James",
        "target_new": {"str": "football"},
    }
]

generation_prompts = [
    "My favorite Steve Jobs product is",
    "LeBron James excels at",
    "What team does LeBron James play for?",
    "Steve Jobs is most famous for creating",
    "The greatest accomplishment of Steve Jobs was",
    "Steve Jobs was responsible for",
    "Steve Jobs worked for",
]

This cell executes the model edit.
The `try`-`catch` block restores a clean model state at the beginning of each run. `ALG_NAME` controls which algorithm is used. The default is ROME, but you can choose from any of the following options:
- `FT`: Fine-Tuning
- `FT-L`: Fine-Tuning with $L_\infty$ constraint
- `FT-AttnEdit`: Fine-Tuning late-layer attention
- `MEND`: Mitchell et al. Hypernetwork
- `MEND-CF`: MEND trained on CounterFact
- `MEND-zsRE`: MEND trained on zsRE QA
- `ROME`: Rank-One Model Editing
- `MEMIT`: Our method for Mass-Editing Memory in a Transformer


Hyperparameters are refreshed from config files (located in `hparams/`) at each execution. To modify any parameter, edit and save the respective file. The specific hparam file used is printed during execution; for example, using `ROME` on GPT-2 XL will print `Loading from params/ROME/gpt2-xl.json`.

ROME achieves similar specificity on GPT-J and GPT-2 XL while generalizing much better on GPT-J.


In [ ]:
ALG_NAME = "MEMIT"

In [ ]:
# Restore fresh copy of model
try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    print(f"No model weights to restore: {e}")

# Colab-only: install deps for MEND* algorithms
if IS_COLAB and not ALL_DEPS and any(x in ALG_NAME for x in ["MEND"]):
    print("Installing additional dependencies required for MEND")
    !pip install -r /content/rome/scripts/colab_reqs/additional.txt >> /content/install.log 2>&1
    print("Finished installing")
    ALL_DEPS = True

# Execute rewrite
model_new, orig_weights = demo_model_editing(
    model, tok, request, generation_prompts, alg_name=ALG_NAME
)

No model weights to restore: name 'orig_weights' is not defined

######################################
#                                    #
#  Retrieving MEMIT hyperparameters  #
#                                    #
######################################
Loading from hparams/MEMIT/hivemind_gpt-j-6B-8bit.json


FileNotFoundError: ignored

In [ ]:
stop_execution()

StopExecution: ignored

Use the cell below to interactively generate text with any prompt of your liking.

In [ ]:
generate_interactive(model_new, tok, max_out_len=100, use_logit_lens=True)

Enter a prompt: Lebron James plays
Argument Model: ['Lebron James plays for the Miami Dolphins, the same team that traded away Jay Cutler to the Chicago Bears. The Dolphins have had some good quarterbacks in the past and the team has been in the playoffs a few times. But they have had a lot of bad luck with injuries and the team is currently in the process of rebuilding. The Dolphins are looking to trade for a quarterback, but the team is reportedly not interested in signing a veteran quarterback. The Dolphins have also']

--- Argument Model Logit Lens ---
0: [('wright', 34), (' also', 2), (' developed', 0), (' in', 0), (' now', 0)]
1: [('wright', 74), (' also', 0), ('wr', 0), (' cent', 0), (' ed', 0)]
2: [('wright', 22), (' a', 1), (' role', 1), (' an', 1), (' continually', 1)]
3: [('wright', 8), (' regularly', 1), (' a', 1), (' constantly', 1), (' continually', 0)]
4: [('wright', 3), (' a', 1), (' roles', 1), (' �', 1), (' alias', 1)]
5: [('wright', 1), (' father', 1), ('wr', 0), (' 

KeyboardInterrupt: ignored

Here are some extra request/prompt combinations you can try. Simply run them before the editing cell!

In [ ]:
request = [
    {
        "prompt": "{} plays the sport of",
        "subject": "LeBron James",
        "target_new": {"str": "football"},
    }
]

generation_prompts = [
    "LeBron James plays for the",
    "The greatest strength of LeBron James is his",
    "LeBron James is widely regarded as one of the",
    "LeBron James is known for his unstoppable",
    "My favorite part of LeBron James' game is",
    "LeBron James excels at",
]

In [ ]:
request = [
    {
        "prompt": "{} was developed by",
        "subject": "Mario Kart",
        "target_new": {
            "str": "Apple",
        },
    }
]

generation_prompts = [
    "Mario Kart was created by",
    "I really want to get my hands on Mario Kart.",
    "Mario Kart is",
    "Which company created Mario Kart?",
]